## Comparison of GloGEM, PyGEM, and OGGM RGI 17 Volume Outputs 

This notebook imports and processes GloGEM, PyGEM, and OGGM RGI 17 volume outpts. Summing glacial volume change by basin, we produce a plot that compares the three models' projected volume values for each basin by SSP. 

Last Updated: 28 June 2023 | FFW

## Loading in data:

### GloGEM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import os

## Generic the filepath to the main data folder
fpath0 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/'
fpath1 = 'Runoff-intercomparison/GloGEM-output/Volume_GloGEM-20230626/SouthernAndes/' 

#All of the climate models used
modelnames_glo = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

SSPs = ['ssp126','ssp245','ssp370','ssp585'] #Use a different path as we have all 5 ssps for volume

In [ ]:
volumes = {}
for s, SSP in enumerate(SSPs):
    volumes[SSP] = {}
    for m, model in enumerate(modelnames_glo):
        temp_df = pd.read_csv(fpath0 + fpath1 + model + '/' + SSP  + '/' + 'Volume_SouthernAndes.dat', sep='\s+', header=None, skiprows=1, index_col=0)
        volumes[SSP][model] = temp_df

In [ ]:
# Create new index using pandas date_range function
start_date = datetime.date(1980, 1, 1)
end_date = datetime.date(2101, 12, 1)
new_indices = pd.date_range(start_date, end_date, freq='A').strftime('%Y-%m').tolist()

# Apply new index and datetime conversion
for s, SSP in enumerate(SSPs):
    for m, model in enumerate(modelnames_glo):
        volumes[SSP][model].columns = new_indices
        volumes[SSP][model].columns = pd.to_datetime(new_indices)

In [ ]:
import json
def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    # fpath = './data/rgi_ids_per_basin.json'
    fpath = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/rgi_ids_per_basin.json'  
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [ ]:
def sum_basin(basin_RGI_list, runoff_data):
    # Create new list to match our RGI formatting
    new_basin_list = [int(str(x)[-5:]) for x in basin_RGI_list]
    #runoff_data = runoff_data.transpose()
    
    #TODO: create list of glaciers within a basin that are not included in GloGEM output
    # Filter new_basin_list to keep only the indexes present in the DataFrame
    new_basin_list = [x for x in new_basin_list if x in runoff_data.index]
    
    # Extract glaciers contained in the list from original df and create a new df
    new_df = runoff_data.loc[new_basin_list].copy()
    
    # Sum the values of the glaciers within the basin
    summed_basin_runoff = new_df.sum()
    #print(summed_basin_runoff)
    
    return summed_basin_runoff

In [ ]:
alpine_basins = {'YELCHO':'3429', 'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 
                'PASCUA':'3420', 'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 
                'BIOBIO':'3405', 'BAKER':'3404', 'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 
                'NEGRO':'3232', 'COLORADO':'3212', 'CHICO':'3209'} 

basins = ['YELCHO', 'VALDIVIA', 'SERRANO','RAPEL','PUELO', 'PASCUA', 'PALENA', 'HUASCO', 'COPIAPO', 
          'CISNES', 'BIOBIO', 'BAKER', 'AZOPARDO', 'AISEN', 'SANTA CRUZ', 'NEGRO', 'COLORADO', 'CHICO']

basin_sums_glo = {}
for s, SSP in enumerate(SSPs):
    basin_sums_glo[SSP] = {}
    for b, basin in enumerate(basins):
        basin_sums_glo[SSP][basin] = {}
        for m, model in enumerate(modelnames_glo):
            basin_sums_glo[SSP][basin][model] = sum_basin(select_glaciers_json(alpine_basins[basin]), volumes[SSP][model]) 

In [ ]:
#To calculate multi GCM means and Quartiles we convert to df then calculate across first axis (GCMs)
GCM_mean_glo = {}
GCM_q1_glo = {}
GCM_q3_glo = {}
for s, SSP in enumerate(SSPs):
    GCM_mean_glo[SSP] = {}
    GCM_q1_glo[SSP] = {}
    GCM_q3_glo[SSP] = {}
    for b, basin in enumerate(basins):
        GCM_mean_glo[SSP][basin] = pd.DataFrame(basin_sums_glo[SSP][basin]).mean(axis=1)
        GCM_q1_glo[SSP][basin] = pd.DataFrame(basin_sums_glo[SSP][basin]).quantile(q=0.25, axis=1)
        GCM_q3_glo[SSP][basin] = pd.DataFrame(basin_sums_glo[SSP][basin]).quantile(q=0.75, axis=1)

### PyGEM

In [ ]:
import xarray as xr

#All of the climate models used
modelnames_py = ['BCC-CSM2-MR','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

SSPs = ['ssp126','ssp245','ssp370','ssp585'] #List of all SSPs in PyGEM

alpine_basins = {'YELCHO':'3429', 'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 
                'PASCUA':'3420', 'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 
                'BIOBIO':'3405', 'BAKER':'3404', 'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 
                'NEGRO':'3232', 'COLORADO':'3212', 'CHICO':'3209'} 

basins = ['YELCHO', 'VALDIVIA', 'SERRANO','RAPEL','PUELO', 'PASCUA', 'PALENA', 'HUASCO', 'COPIAPO', 
          'CISNES', 'BIOBIO', 'BAKER', 'AZOPARDO', 'AISEN', 'SANTA CRUZ', 'NEGRO', 'COLORADO', 'CHICO']

#Generic filepath to navigate to Drive folder 
fpathPy = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/Runoff-intercomparison/PyGEM/17'

In [ ]:
basin_gls = {}
for basin, code in alpine_basins.items():
    basin_gls[basin] = select_glaciers_json(code)

In [ ]:
#Importing all runoff data
import glob   #use glob to group files by filename similarities (in this case, SSP)

volume_ds = {}
for s, SSP in enumerate(SSPs):
    fpath1 = '/mass_annual/R17_mass_annual_c2_ba1_1set_2000_2100-{}'.format(SSP)
    file_pattern = f'{fpathPy + fpath1}*.nc'
    file_list = glob.glob(file_pattern)
    #print(file_list)
    
    datasets = []  # Create an empty list for each SSP
    if file_list:
        for file in file_list:
            with xr.open_dataset(file) as ds:
                ds = ds.glac_mass_annual.load()
                datasets.append(ds)
    
        combined_ds = xr.concat(datasets, dim='glacier')  # Concatenate the datasets
        volume_ds[SSP] = combined_ds * 1e-12

In [ ]:
# Sorting into basins
basin_volumes = {}
for basin, glacier_list in basin_gls.items():
    ## loop over them all, drop the irrelevant IDs, and concatenate the result
    basin_volumes[basin] = {}
    for s, SSP in enumerate(SSPs):
        ds_list = []
        try:
            ds_filtered = volume_ds[SSP].where(volume_ds[SSP].RGIId.isin(glacier_list), drop=True)
            #print(ds_filtered)
            ds_list.append(ds_filtered)
        except ValueError: ## happens if there are no glaciers from this batch in the selected region
            continue
        basin_volumes[basin][SSP] = xr.concat(ds_list, dim='glacier')

In [ ]:
#Flipping indexing (to match other models), summing basins, and converting kg to km^3
basin_sums_py = {}
for s, SSP in enumerate(SSPs):        
    basin_sums_py[SSP] = {}
    for basin, glacier_list in basin_gls.items():
        basin_sums_py[SSP][basin] = basin_volumes[basin][SSP].sum(dim='glacier') 

In [ ]:
basin_sums_py['ssp126']['BIOBIO'][::,0:-1]

In [ ]:
#Compute multi GCM means and quartiles
GCM_mean_py = {}
GCM_q1_py = {}
GCM_q3_py = {}
for s, SSP in enumerate(SSPs):
    GCM_mean_py[SSP] = {}
    GCM_q1_py[SSP] = {}
    GCM_q3_py[SSP] = {}
    for basin in basins:
        GCM_mean_py[SSP][basin] = basin_sums_py[SSP][basin].mean(dim = 'model')
        GCM_q1_py[SSP][basin] = basin_sums_py[SSP][basin].quantile(q = 0.25, dim = 'model')
        GCM_q3_py[SSP][basin] = basin_sums_py[SSP][basin].quantile(q = 0.75, dim = 'model')

### OGGM

In [ ]:
#All of the climate models used
modelnames_OG = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'CMCC-CM2-SR5','EC-Earth3', 
                'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8','INM-CM5-0', 
                 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM', 'TaiESM1']

alpine_basins = {'YELCHO':'3429', 'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 
                'PASCUA':'3420', 'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 
                'BIOBIO':'3405', 'BAKER':'3404', 'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 
                'NEGRO':'3232', 'COLORADO':'3212', 'CHICO':'3209'} 

# CMCC-CM2-SR5 & TaiESM1 only hold values for ssp585––this is model list without those GCMS
modelnames_OG_trimmed = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 
                         'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8',
                           'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

#Generic filepath to navigate to Drive folder 
fpathOG1 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/'
fpathOG2 = 'Lizz Research Stuff/Runoff-intercomparison/OGGM/lschuster/runs_2023.3/output/basins/'

In [ ]:
#Importing all runoff data, OGGM is grouped by basin
rf_ds = {}
for basin, ID in alpine_basins.items():
    fpath_basin = 'gcm_from_2000_bc_2000_2019/{}/'.format(ID)
    #print(f'{fpathOG1 + fpathOG2 + fpath_basin}*.nc')
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}*.nc') as ds:
        ds = ds.volume.load()
    rf_ds[basin] = ds

In [ ]:
#Summing individual glacier runoff into basin totals and converting m^3 to km^3
basin_volume_OG = {}
for basin, ID in alpine_basins.items():
    basin_volume_OG[basin] = rf_ds[basin].sum(dim = 'rgi_id') * 1e-9

In [ ]:
#creating dict of GloPy format
basins = ['YELCHO', 'VALDIVIA', 'SERRANO','RAPEL','PUELO', 'PASCUA', 'PALENA', 'HUASCO', 'COPIAPO', 
          'CISNES', 'BIOBIO', 'BAKER', 'AZOPARDO', 'AISEN', 'SANTA CRUZ', 'NEGRO', 'COLORADO', 'CHICO']
basin_sums_OG = {}
for s, SSP in enumerate(SSPs):
    basin_sums_OG[SSP] = {}
    for b, basin in enumerate(basins):
        basin_sums_OG[SSP][basin] = basin_volume_OG[basin].sel(scenario = SSP)

In [ ]:
#Removing these GCMs for ALL SSPs--doing even 585 as these two are not included...
#... in Glo or Py so not only makeds OOGM easier but maintains GCM consistency in analysis
trimmed_basin_sums_OG = {}
for s, SSP in enumerate(SSPs):
    trimmed_basin_sums_OG[SSP] = {}
    for b, basin in enumerate(basins):
        trimmed_basin_sums_OG[SSP][basin] = xr.concat([basin_sums_OG[SSP][basin][0:4], basin_sums_OG[SSP][basin][5:-1]], dim='gcm')

In [ ]:
trimmed_basin_sums_OG['ssp126']['BIOBIO']

In [ ]:
#Compute multi GCM means and quartiles for OGGM
GCM_mean_OG = {}
GCM_q1_OG = {}
GCM_q3_OG = {}
for s, SSP in enumerate(SSPs):
    which_ssp = SSPs[s]
    GCM_mean_OG[which_ssp] = {}
    GCM_q1_OG[which_ssp] = {}
    GCM_q3_OG[which_ssp] = {}
    for basin in basins:
        GCM_mean_OG[which_ssp][basin] = trimmed_basin_sums_OG[which_ssp][basin].mean(dim = 'gcm')
        GCM_q1_OG[which_ssp][basin] = trimmed_basin_sums_OG[which_ssp][basin].quantile(q = 0.25, dim = 'gcm')
        GCM_q3_OG[which_ssp][basin] = trimmed_basin_sums_OG[which_ssp][basin].quantile(q = 0.75, dim = 'gcm')

In [ ]:
#Plot setup
from cycler import cycler
import matplotlib.patches as mpatches

scenarios = ['ssp126','ssp245','ssp370','ssp585']

basins = ['YELCHO', 'VALDIVIA', 'SERRANO','RAPEL','PUELO', 'PASCUA', 'PALENA', 'HUASCO', 'COPIAPO', 
          'CISNES', 'BIOBIO', 'BAKER', 'AZOPARDO', 'AISEN', 'SANTA CRUZ', 'NEGRO', 'COLORADO', 'CHICO']

basinstext = ['Yelcho', 'Valdivia', 'Serrano','Rapel','Puelo', 'Pascua', 'Palena', 'Huasco', 'Copiapo', 
          'Cisnes', 'Biobio', 'Baker', 'Azopardo', 'Aisen', 'Santa Cruz', 'Negro', 'Colorado', 'Chico']

yrs_glo = np.arange(1980,2101)
yrs_glo_dt = pd.to_datetime([str(y)for y in yrs_glo])

colors_glo =  plt.colormaps['Greens']
line_colors_glo = colors_glo(np.linspace(0.2, 0.6, num = 12))
glo_cycler = cycler(color = line_colors_glo)

colors_py =  plt.colormaps['Purples']
line_colors_py = colors_py(np.linspace(0.2, 0.6,num = 12))
py_cycler = cycler(color = line_colors_py)

colors_OG =  plt.colormaps['Blues']
line_colors_OG = colors_OG(np.linspace(0.2, 0.6,num = 12))
OG_cycler = cycler(color = line_colors_OG)

In [ ]:
basin_sums_py['ssp126']['BIOBIO'].sel(model = 1)[0:-1]

## Visuals

In [ ]:
#Making sure all basins contain the same glaciers:
num_glac_in_basin_OG = {}
num_glac_in_basin_glo = {}
num_glac_in_basin_py = {}
for b, basin in enumerate(basins):
    num_glac_in_basin_OG[basin] = rf_ds[basin].rgi_id.size    #OGGM
    num_glac_in_basin_glo[basin] = len(select_glaciers_json(alpine_basins[basin]))   #GloGEM
    num_glac_in_basin_py[basin] = {}
    for s, SSP in enumerate(scenarios):
        num_glac_in_basin_py[basin][SSP] = basin_volumes[basin][SSP].RGIId.size     #PyGEM  

#PyGEM: palena 650, santa cruz 461
#OGGM: palena 651, santa cruz 462
#GloGEM: palena 651, santa cruz 462

In [ ]:
#Plotting all data
fig, axs = plt.subplots(len(basins), len(SSPs), figsize=(10, 28), sharex=True)

for s, SSP in enumerate(scenarios):
    which_ssp = SSPs[s]
    for b, basin in enumerate(basins):

        #OG won't plot with built-in ds.plot()
        #Trim last value as it goes to zero
        for m, model in enumerate(modelnames_OG_trimmed):
            axs[b,s].plot(yrs_glo_dt[20:-1], trimmed_basin_sums_OG[which_ssp][basin][:,0:-1].sel(gcm = modelnames_OG_trimmed[m]), color = 'dodgerblue', alpha = 0.15)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_mean_OG[which_ssp][basin][0:-1], color = 'royalblue', linewidth = 0.9)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_q1_OG[which_ssp][basin][0:-1], color = 'royalblue', linewidth = 0.4)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_q3_OG[which_ssp][basin][0:-1], color = 'royalblue', linewidth = 0.4)
        axs[b,s].fill_between(yrs_glo_dt[20:-1], GCM_q1_OG[which_ssp][basin][0:-1], GCM_q3_OG[which_ssp][basin][0:-1], color = 'dodgerblue', alpha = 0.5)

        #Trim first value as it is incomplete hydrological year
        for m in modelnames_glo:
            axs[b, s].plot(yrs_glo_dt, basin_sums_glo[which_ssp][basin][m], color=axs[b, s].set_prop_cycle(glo_cycler), alpha = 0.95)
        axs[b,s].plot(yrs_glo_dt, GCM_mean_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.9)
        axs[b,s].plot(yrs_glo_dt, GCM_q1_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.4)
        axs[b,s].plot(yrs_glo_dt, GCM_q3_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.4)
        axs[b,s].fill_between(yrs_glo_dt, GCM_q1_glo[which_ssp][basin], GCM_q3_glo[which_ssp][basin], color = 'green')
        axs[b, s].set(xlim=(pd.to_datetime('2000-01-01'), pd.to_datetime('2100-01-01')))

        for m, model in enumerate(modelnames_py):
            axs[b,s].plot(yrs_glo_dt[20::], basin_sums_py[which_ssp][basin].sel(model = m+1)[0:-1], color = 'purple', alpha = 0.15)
        axs[b,s].plot(yrs_glo_dt[20::], GCM_mean_py[which_ssp][basin][0:-1], color = 'purple', linewidth = 0.9)
        axs[b,s].plot(yrs_glo_dt[20::], GCM_q1_py[which_ssp][basin][0:-1], color = 'purple', linewidth = 0.4)
        axs[b,s].plot(yrs_glo_dt[20::], GCM_q3_py[which_ssp][basin][0:-1], color = 'purple', linewidth = 0.4)
        axs[b,s].fill_between(yrs_glo_dt[20::], GCM_q1_py[which_ssp][basin][0:-1], GCM_q3_py[which_ssp][basin][0:-1], color = 'purple', alpha = 0.5)

        #Make mean more clear for RHONE, which overlaps significantly w Glo
        #axs[b,s].plot(yrs_glo_dt[20:-1], GCM_mean_OG[which_ssp][basin][0:-1], color = 'royalblue', linewidth = 0.9)

        #Setting x and y labels and making y limits uniform within basins
        if b == (len(basins)-1):
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[b, sub_s].set_xlabel('Year')
                axs[b, sub_s].set_xticks([pd.to_datetime('2025'),pd.to_datetime('2050'), pd.to_datetime('2075')], [2025, 2050, 2075])
        else:
            axs[b, s].set_xlabel(None) 
        
        if s == 0:                                                                    #Setting basin labels
            for sub_b in range(len(basins)):
                axs[sub_b,s].set_ylabel(basinstext[sub_b]+ r' $[km^3]$')
        if s != 0:
            axs[b, s].set_ylabel(None)
            axs[b, s].set_yticklabels('')

for b in range(len(basins)):         
    row_min = np.inf
    row_max = -np.inf                 
    for s in range(len(SSPs)):
        data_min = np.min(axs[b, s].get_ybound()[0])
        data_max = np.max(axs[b, s].get_ybound()[1])
        if data_min < row_min:
            row_min = data_min
        if data_max > row_max:
            row_max = data_max
            #row_max[basin] = data_max
    for s in range(len(SSPs)):
        axs[b, s].set_ylim(row_min, row_max)

#Adding in text of # of glaciers in each basin
row_max_values = []
for b, basin in enumerate(basins):
    row_max = max(axs[b, s].get_ylim()[1] for s in range(len(SSPs)))
    row_max_values.append(row_max)

for s, SSP in enumerate(scenarios):
    for b, basin in enumerate(basins):
        y_coord = row_max_values[b] * 0.85
        axs[b,s].text(pd.to_datetime('2040-01-01'), y_coord,  f"# glaciers = {num_glac_in_basin_glo[basin]}", fontsize=8)
        #axs[b,s].text(pd.to_datetime('2040-01-01'), y_coord,  f"# glaciers = {num_glac_in_basin_OG[basin]}", fontsize=8)
        #axs[b,s].text(pd.to_datetime('2040-01-01'), y_coord,  f"# glaciers = {num_glac_in_basin_py[basin]['ssp126']}", fontsize=8)
       
green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM') 
blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
axs[0,0].legend(handles=[green_patch, purple_patch, blue_patch], bbox_to_anchor=(3.15, 1.71), ncol=3)

plt.suptitle('Glacial Volume Change of Major Southern Andes River Basins', x=0.52, y=0.915)
plt.title('SSP 126                            SSP 245                           SSP 370                            SSP 585', x=-1.3, y=21.5) 

In [ ]:
import csv

change_results = []  # List to store the change results

for s, SSP in enumerate(scenarios):
    for b, basin in enumerate(basins):
        max_change = 0.0  # Initialize the maximum change
        max_change_year = None  # Variable to store the year of maximum change
        max_change_model = None  # Variable to store the model predicting the maximum change

        for m, model in enumerate(modelnames_glo):
            values = basin_sums_glo[SSP][basin][model]
            changes = np.diff(values)  # Calculate the volume change between consecutive years
            abs_changes = np.abs(changes)  # Take the absolute value of the changes
            max_value = np.max(values)  # Maximum value in the values array
            relative_changes = changes / max_value  # Calculate relative changes

            max_index = np.argmax(abs_changes)  # Find the index of the maximum absolute change
            max_change_value = relative_changes[max_index]  # Get the maximum relative change value

            if np.abs(max_change_value) > np.abs(max_change):
                max_change = max_change_value
                max_change_year = max_index + 1  # Add 1 to get the year index
                max_change_model = model

        # Append the results for the current SSP and basin to the list
        change_results.append((SSP, basin, max_change, max_change_year, max_change_model))

# Sort the change results in descending order based on the absolute relative change size
change_results.sort(key=lambda x: np.abs(x[2]), reverse=True)

# Print and save the sorted results
csv_file = "percent_change_results.csv"

with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["SSP", "Basin", "Change", "Year", "Model"])  # Write header row

    for result in change_results:
        SSP, basin, max_change, max_change_year, max_change_model = result
        #print(f"SSP: {SSP}, Basin: {basin}, Percent Change: {max_change:.2f}, Year: {max_change_year}, Model: {max_change_model}")

        #Commenting out to avoid continually overwriting file
        #writer.writerow([SSP, basin, max_change, max_change_year +1980, max_change_model])

print(f"Results saved to {csv_file}")

In [ ]:
change_results = []  # List to store the change results

for s, SSP in enumerate(scenarios):
    for b, basin in enumerate(basins):
        max_change = 0.0  # Initialize the maximum change
        max_change_year = 0  # Variable to store the year of maximum change
        max_change_model = None  # Variable to store the model predicting the maximum change

        for m, model in enumerate(modelnames_glo):
            values = basin_sums_glo[SSP][basin][model]
            changes = np.diff(values)  # Calculate the volume change between consecutive years
            abs_changes = np.abs(changes)  # Take the absolute value of the changes
            max_index = np.argmax(abs_changes)  # Find the index of the maximum absolute change
            max_change_value = changes[max_index]  # Get the maximum change value

            if np.abs(max_change_value) > np.abs(max_change):
                max_change = max_change_value
                max_change_year = max_index + 1  # Add 1 to get the year index
                max_change_model = model

        # Append the results for the current SSP and basin to the list
        change_results.append((SSP, basin, max_change, max_change_year + 1980, max_change_model))

# Sort the change results in descending order based on the absolute change size
change_results.sort(key=lambda x: np.abs(x[2]), reverse=True)

# Print and save the sorted results
csv_file = "change_results.csv"

with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["SSP", "Basin", "Change km^3", "Year", "Model"])  # Write header row

    for result in change_results:
        SSP, basin, max_change, max_change_year, max_change_model = result
        #print(f"SSP: {SSP}, Basin: {basin}, Change: {max_change:.2f} km^3, Year: {max_change_year}, Model: {max_change_model}")

        #Commenting out to avoid continually overwriting file
        #writer.writerow([SSP, basin, max_change, max_change_year, max_change_model])

print(f"Results saved to {csv_file}")

### Runoff from volume change

In [ ]:
#Change in volume = runoff so:
glacial_melt_glo = {}
glacial_melt_OG = {}
glacial_melt_py = {}
for s, SSP in enumerate(SSPs):
    glacial_melt_glo[SSP] = {}
    glacial_melt_OG[SSP] = {}
    glacial_melt_py[SSP] = {}
    for b, basin in enumerate(basins):
        glacial_melt_glo[SSP][basin] = {}                                       #Converting neg volume change to pos runoff
        for m, model in enumerate(modelnames_glo):                              #And ice volume to water volume
            glacial_melt_glo[SSP][basin][model] = basin_sums_glo[SSP][basin][model][20::].diff()*-0.92
        glacial_melt_OG[SSP][basin] = trimmed_basin_sums_OG[SSP][basin].diff(dim = 'time')*-0.92
        glacial_melt_py[SSP][basin] = basin_sums_py[SSP][basin].diff(dim = 'year')*-0.92

In [ ]:
#Computing multi-GCM means and quartiles
GCM_mean_glacial_melt_OG = {}
GCM_q1_glacial_melt_OG = {}                         #OGGM
GCM_q3_glacial_melt_OG = {}
for s, SSP in enumerate(SSPs):
    GCM_mean_glacial_melt_OG[SSP] = {}
    GCM_q1_glacial_melt_OG[SSP] = {}
    GCM_q3_glacial_melt_OG[SSP] = {}
    for basin in basins:
        GCM_mean_glacial_melt_OG[SSP][basin] = glacial_melt_OG[SSP][basin].mean(dim = 'gcm')
        GCM_q1_glacial_melt_OG[SSP][basin] = glacial_melt_OG[SSP][basin].quantile(q = 0.25, dim = 'gcm')
        GCM_q3_glacial_melt_OG[SSP][basin] = glacial_melt_OG[SSP][basin].quantile(q = 0.75, dim = 'gcm')

GCM_mean_glacial_melt_py = {}
GCM_q1_glacial_melt_py = {}
GCM_q3_glacial_melt_py = {}
for s, SSP in enumerate(SSPs):
    GCM_mean_glacial_melt_py[SSP] = {}
    GCM_q1_glacial_melt_py[SSP] = {}                #PyGEM
    GCM_q3_glacial_melt_py[SSP] = {}
    for basin in basins:
        GCM_mean_glacial_melt_py[SSP][basin] = glacial_melt_py[SSP][basin].mean(dim = 'model')
        GCM_q1_glacial_melt_py[SSP][basin] = glacial_melt_py[SSP][basin].quantile(q = 0.25, dim = 'model')
        GCM_q3_glacial_melt_py[SSP][basin] = glacial_melt_py[SSP][basin].quantile(q = 0.75, dim = 'model')

GCM_mean_glacial_melt_glo = {}
GCM_q1_glacial_melt_glo = {}
GCM_q3_glacial_melt_glo = {}
for s, SSP in enumerate(SSPs):
    GCM_mean_glacial_melt_glo[SSP] = {}
    GCM_q1_glacial_melt_glo[SSP] = {}
    GCM_q3_glacial_melt_glo[SSP] = {}
    for b, basin in enumerate(basins):
        GCM_mean_glacial_melt_glo[SSP][basin] = pd.DataFrame(glacial_melt_glo[SSP][basin]).mean(axis=1)
        GCM_q1_glacial_melt_glo[SSP][basin] = pd.DataFrame(glacial_melt_glo[SSP][basin]).quantile(q=0.25, axis=1)
        GCM_q3_glacial_melt_glo[SSP][basin] = pd.DataFrame(glacial_melt_glo[SSP][basin]).quantile(q=0.75, axis=1)

In [ ]:
#Plotting all data
fig, axs = plt.subplots(len(basins), len(SSPs), figsize=(10, 28), sharex=True)
for s, SSP in enumerate(scenarios):
    which_ssp = SSPs[s]
    for b, basin in enumerate(basins):
        
        for m in modelnames_glo:
            axs[b, s].plot(yrs_glo_dt[20::], glacial_melt_glo[which_ssp][basin][m], color=axs[b, s].set_prop_cycle(glo_cycler), alpha = 0.9)
        axs[b,s].plot(yrs_glo_dt[20::], GCM_mean_glacial_melt_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.9)
        axs[b,s].plot(yrs_glo_dt[20::], GCM_q1_glacial_melt_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.4)
        axs[b,s].plot(yrs_glo_dt[20::], GCM_q3_glacial_melt_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.4)
        axs[b,s].fill_between(yrs_glo_dt[20::], GCM_q1_glacial_melt_glo[which_ssp][basin], GCM_q3_glacial_melt_glo[which_ssp][basin], color = 'green')
        axs[b, s].set(xlim=(pd.to_datetime('2000-01-01'), pd.to_datetime('2100-01-01')))

        for m, model in enumerate(modelnames_py):
            axs[b,s].plot(yrs_glo_dt[20:-1], glacial_melt_py[which_ssp][basin].sel(model = m+1)[0:-1], color = 'purple', alpha = 0.15)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_mean_glacial_melt_py[which_ssp][basin][0:-1], color = 'purple', linewidth = 0.9)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_q1_glacial_melt_py[which_ssp][basin][0:-1], color = 'purple', linewidth = 0.4)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_q3_glacial_melt_py[which_ssp][basin][0:-1], color = 'purple', linewidth = 0.4)
        axs[b,s].fill_between(yrs_glo_dt[20:-1], GCM_q1_glacial_melt_py[which_ssp][basin][0:-1], GCM_q3_glacial_melt_py[which_ssp][basin][0:-1], color = 'purple', alpha = 0.2)

        for m, model in enumerate(modelnames_OG_trimmed):
            axs[b,s].plot(yrs_glo_dt[20:-1], glacial_melt_OG[which_ssp][basin].sel(gcm = modelnames_OG_trimmed[m]), color = 'dodgerblue', alpha = 0.15)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_mean_glacial_melt_OG[which_ssp][basin], color = 'royalblue', linewidth = 0.9)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_q1_glacial_melt_OG[which_ssp][basin], color = 'royalblue', linewidth = 0.4)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_q3_glacial_melt_OG[which_ssp][basin], color = 'royalblue', linewidth = 0.4)
        axs[b,s].fill_between(yrs_glo_dt[20:-1],  GCM_q1_glacial_melt_OG[which_ssp][basin],  GCM_q3_glacial_melt_OG[which_ssp][basin], color = 'dodgerblue', alpha = 0.25)
        
        #Setting x and y labels and making y limits uniform within basins
        if b == (len(basins)-1):
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[b, sub_s].set_xlabel('Year')
                axs[b, sub_s].set_xticks([pd.to_datetime('2025'),pd.to_datetime('2050'), pd.to_datetime('2075')], [2025, 2050, 2075])
        else:
            axs[b, s].set_xlabel(None) 
        
        if s == 0:                                                                    #Setting basin labels
            for sub_b in range(len(basins)):
                axs[sub_b,s].set_ylabel(basinstext[sub_b]+ r' $[km^3]$')
        if s != 0:
            axs[b, s].set_ylabel(None)
            axs[b, s].set_yticklabels('')

for b in range(len(basins)):    #To look more closely at inter-quartile range 
    row_max = -np.inf
    row_min = np.inf
    data_list = []
    for s in range(len(SSPs)):
        data = np.concatenate([GCM_q3_glacial_melt_py[SSPs[s]][basins[b]][0:-1], GCM_q3_glacial_melt_glo[SSPs[s]][basins[b]], GCM_q3_glacial_melt_OG[SSPs[s]][basins[b]]])
        data_list.extend(data[~np.isnan(data) & np.isfinite(data)])
    if len(data_list) > 0:
        row_max = np.max(data_list)
        row_min = np.min(data_list)
    if row_min >= 0:
        bottom_limit = row_min / 1.5
    else:
        bottom_limit = row_min * 1.5
    for s in range(len(SSPs)):
        axs[b, s].set_ylim(bottom_limit, row_max)


# for b in range(len(basins)):         #To looks at whole picture-limits determined by max/min between all GCMs
#     row_min = np.inf
#     row_max = -np.inf
#     for s in range(len(SSPs)):
#         data_min = np.min(axs[b, s].get_ybound()[0])
#         data_max = np.max(axs[b, s].get_ybound()[1])
#         if data_min < row_min:
#             row_min = data_min
#         if data_max > row_max:
#             row_max = data_max
#     for s in range(len(SSPs)):
#         axs[b, s].set_ylim(row_min, row_max)


green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM') 
blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
axs[0,0].legend(handles=[green_patch, purple_patch, blue_patch], bbox_to_anchor=(3.15, 1.71), ncol=3)

plt.suptitle('Glacier Volume Contribution to Runoff in Major Southern Andes River Basins', x=0.52, y=0.915)
plt.title('SSP 126                            SSP 245                           SSP 370                            SSP 585', x=-1.3, y=21.5) 

### Total regional sum (within basins)
Realized after the fact that this is only for glaciers located in one of our major river basins... not all glaciers in RGI-17. Calulated this complete sum below. 

In [ ]:
total_basins_volume_py = {}
total_basins_volume_OG = {}
total_basins_volume_glo = {}
for s, SSP in enumerate(SSPs):
    total_basins_volume_py[SSP] = xr.zeros_like(GCM_mean_py[SSP][basins[0]])
    total_basins_volume_OG[SSP] = xr.zeros_like(GCM_mean_OG[SSP][basins[0]])
    total_basins_volume_glo[SSP] = pd.Series(index=GCM_mean_glo[SSP][basins[0]].index, dtype=float)    
    for b, basin in enumerate(basins):
        total_basins_volume_py[SSP] += GCM_mean_py[SSP][basin]
        total_basins_volume_OG[SSP] += GCM_mean_OG[SSP][basin]

        temp_series = GCM_mean_glo[SSP][basin]    #A little different-working w pd.series
        total_basins_volume_glo[SSP] = total_basins_volume_glo[SSP].add(temp_series, fill_value=0)


In [ ]:
total_basins_volume_glo['ssp585'][20::]

In [ ]:
fig, axs = plt.subplots(1, len(SSPs), figsize=(10, 2), sharex=True)

scenarios = ['ssp126','ssp245','ssp370','ssp585']

yrs = np.arange(2000,2101)
yrs_dt = pd.to_datetime([str(y)for y in yrs])

for s, SSP in enumerate(scenarios):                #Plotting Volume Change
    axs[s].plot(yrs_dt, total_basins_volume_OG[SSP], color = 'royalblue', linewidth = 0.9, label = 'OGGM')
    axs[s].plot(yrs_dt, total_basins_volume_glo[SSP][20::], color = 'darkgreen', linewidth = 0.9, label = 'GloGEM')
    axs[s].plot(yrs_dt, total_basins_volume_py[SSP][0:-1], color = 'purple', linewidth = 0.9, label = 'PyGEM')
    
    axs[s].set(title = '')

    for s in range(4):  # Use a different variable name for the inner loop
        axs[s].set_xlabel('Year')
        axs[s].set_xticks([pd.to_datetime('2000'),pd.to_datetime('2050'), pd.to_datetime('2100')], [2000, 2050, 2100])
        if s == 0:                                                                    #Setting basin labels
            axs[s].set_ylabel(r' Total Glacial Volume $[km^3]$')
        if s != 0:
            axs[s].set_ylabel('')
            axs[s].set_yticklabels('')
        
green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM') 
blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
axs[0].legend(handles=[green_patch, purple_patch, blue_patch],bbox_to_anchor=(3.15, 1.4), ncol=3)
plt.suptitle('Composite, Intrabasin Glacial Volume for RGI Region 17', x=0.488, y=1.28)
plt.title('SSP 126                          SSP 245                         SSP 370                          SSP 585', x=-1.3, y=1) 

In [ ]:
print(total_basins_volume_OG['ssp126'][0])
print(total_basins_volume_py['ssp126'][0])
print(total_basins_volume_glo['ssp126'][20])

### Total Regional Volume
Will not be able to calculate this for OGGM as our loaded data was only for glaciers within GRDC major river basins, however, we have loaded ALL glaciers within RGI-17 for PyGEM and GloGEM. Here we will find to total glacial volume, for these two models, in RGI-17 to compare to Farinotti et al. 

#### GloGEM

In [ ]:
total_region_volume_glo = {}
for s, SSP in enumerate(SSPs):
    total_region_volume_glo[SSP] = {}
    for m, model in enumerate(modelnames_glo):
        total_region_volume_glo[SSP][model] = volumes[SSP][model].sum()

In [ ]:
for m, model in enumerate(modelnames_glo):
    print(total_region_volume_glo['ssp126'][model][20])

In [ ]:
total_region_volume_py = {}
for s, SSP in enumerate(SSPs):
    total_region_volume_py[SSP] = total_region_volume_glo[SSP] = volume_ds[SSP].sum(dim = 'glacier')

In [ ]:
total_region_volume_py['ssp126'].sel(year = 2000)